In [47]:
#Import packages

#Rdkit
from rdkit import Chem

#Scipy
from scipy import stats 
from scipy.stats import uniform,ranksums 

#Pandas
import pandas as pd
from pandas import DataFrame

#AlvaDesc
from alvadesccliwrapper.alvadesc import AlvaDesc

#Others
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import joblib #jbolib模块
import random
import numpy as np
import os
import math
import warnings

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from plotnine import *

In [56]:
import torch
import deepchem as dc
import numpy as np

In [59]:
def print_RxC(dataframe):
    rows= dataframe.shape[0]
    columns= dataframe.shape[1]
    rows_columns=print(dataframe.shape[0],"Moleculars ×",dataframe.shape[1],"Descriptors")
    return rows_columns
def trimm_correlated(df_in, threshold):
    df_corr = df_in.corr(method='spearman')
    df_not_correlated = ~(df_corr.mask(np.tril(np.ones([len(df_corr)]*2, dtype=bool))).abs() > threshold).any()
    un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
    df_out = df_in[un_corr_idx]
    return df_out

In [2]:
#Setting the Work Path
warnings.filterwarnings ('ignore')
#Work Path
os.chdir("C:\\Users\\A\\Desktop\\Bioactive\\HSV-1")

In [9]:
Raw_data1= pd.read_csv('./AID_588824_datatable_all.csv',index_col=0)
Raw_data2= pd.read_csv('./AID_651615_datatable_all.csv',index_col=0)
Raw_data3= pd.read_csv('./AID_687007_datatable_all.csv',index_col=0)

In [21]:
df = pd.concat([Raw_data1,Raw_data2,Raw_data3])
df.dropna(subset=["PUBCHEM_CID"], inplace=True)  
df['Activity']=np.where(df['PUBCHEM_ACTIVITY_OUTCOME']=='Active', 1, 0)
df.sort_values(by="Activity" , inplace=True, ascending=False) 
df.drop_duplicates(subset='PUBCHEM_CID', keep='first', inplace=True)
df['cid']=df['PUBCHEM_CID']
df.index=df['cid']
df=df.drop(['PUBCHEM_CID'], axis=1)
df=df.drop(['PUBCHEM_SID'], axis=1)
df=df.drop(['cid'], axis=1)


,PUBCHEM_EXT_DATASOURCE_SMILES,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,PUBCHEM_ACTIVITY_URL,PUBCHEM_ASSAYDATA_COMMENT,Average Inhibition at 3.6 uM,Standard Deviation,Inhibition at 3.6 uM [1],Inhibition at 3.6 uM [2],Inhibition at 3.6 uM [3],Inhibition at 3.6 uM [4],Average Inhibition at 8.5 uM,Inhibition at 8.5 uM [1],Inhibition at 8.5 uM [2],Inhibition at 8.5 uM [3],Activity
cid,,,,,,,,,,,,,,,,
4366047.0,CC1C(C(C(C(O1)NC2=NC=NC3=C2NC=N3)O)O)NC(=O)CNC...,Active,100,NaN,NaN,98.78,0.214488,98.501800,99.03020,98.942200,98.6469,NaN,NaN,NaN,NaN,1
240739.0,C1=CC2=C(C=C1S(=O)(=O)N)C(=O)C(=C(C2=O)Cl)N,Active,96,NaN,NaN,95.79,0.927476,96.489800,94.47520,96.392000,NaN,NaN,NaN,NaN,NaN,1
619061.0,C1OC2=C(O1)C(=C3C(=C2)C4=CC(C(C(C4NC3=O)O)O)O)O,Active,96,NaN,NaN,96.02,0.472304,96.630000,95.94620,95.480000,NaN,NaN,NaN,NaN,NaN,1
275781.0,C1=CN(C2=NC=NC(=C21)NO)C3C(C(C(O3)CO)O)O,Active,96,NaN,NaN,96.00,0.146850,95.882300,95.91270,96.207900,NaN,NaN,NaN,NaN,NaN,1
9661009.0,CC1=CC=C(C=C1)S(=O)(=O)/N=C/2\C=C(C(=O)C3=CC=C...,Active,96,NaN,NaN,95.99,0.256400,95.679800,96.30770,95.982500,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135449258.0,COC1=CC=C(C=C1)NC(=O)CSC2=NC(=CC(=O)N2C3=CC=CC...,Inactive,2,NaN,NaN,2.37,4.938570,-0.435077,-1.76342,9.313690,NaN,NaN,NaN,NaN,NaN,0
2048059.0,COC1=CC=C(C=C1)NC(=O)CSC2=NC=NC3=C2C=NN3CC4=CC...,Inactive,2,NaN,NaN,2.38,2.341030,2.038150,5.40147,-0.302383,NaN,NaN,NaN,NaN,NaN,0
13354024.0,CC1(CC(=CC(=O)C1)C2=CC3=C(C=C2)OCO3)C,Inactive,2,NaN,NaN,2.47,5.243350,-4.639300,4.22086,7.843130,NaN,NaN,NaN,NaN,NaN,0


In [25]:
df['IsomericSMILES']=df['PUBCHEM_EXT_DATASOURCE_SMILES']
Original_data=pd.DataFrame(df['IsomericSMILES'])
Original_data['Activity']=df['Activity']
Chemdraw_smiles=Original_data['IsomericSMILES'].tolist()

Original_data.head()

,IsomericSMILES,Activity
cid,,
4366047.0,CC1C(C(C(C(O1)NC2=NC=NC3=C2NC=N3)O)O)NC(=O)CNC...,1
240739.0,C1=CC2=C(C=C1S(=O)(=O)N)C(=O)C(=C(C2=O)Cl)N,1
619061.0,C1OC2=C(O1)C(=C3C(=C2)C4=CC(C(C(C4NC3=O)O)O)O)O,1
275781.0,C1=CN(C2=NC=NC(=C21)NO)C3C(C(C(O3)CO)O)O,1
9661009.0,CC1=CC=C(C=C1)S(=O)(=O)/N=C/2\C=C(C(=O)C3=CC=C...,1


In [36]:
Smis=[]
for i in Chemdraw_smiles:
    Smis.append(i)
Mols = []
for Smi in Smis:
    Mol = Chem.MolFromSmiles(Smi)
    Mols.append(Mol)
Canonical_smis = []
for Mol in Mols:
    Canonical_smi = Chem.MolToSmiles(Mol)
    Canonical_smis.append(Canonical_smi)
Original_data['Canonical_smiles']=Canonical_smis
Original_data.to_csv("./Original_data.csv",sep=',',header=1,index=1)
Original_data

,IsomericSMILES,Activity,Canonical_smiles
cid,,,
596.0,C1=CN(C(=O)N=C1N)C2C(C(C(O2)CO)O)O,0,CC(C)CCCCCCCCCCCCC(=O)NCC(=O)NC1C(C)OC(Nc2ncnc...
1318.0,C1=CC2=C(C3=C(C=CC=N3)C=C2)N=C1,1,NC1=C(Cl)C(=O)c2ccc(S(N)(=O)=O)cc2C1=O
1570.0,C1=CC(=S)N(C=C1)O,1,O=C1NC2C(=CC(O)C(O)C2O)c2cc3c(c(O)c21)OCO3
1821.0,C1=C(C(=O)NC(=O)N1C2C(C(C(O2)CO)O)O)F,1,OCC1OC(n2ccc3c(NO)ncnc32)C(O)C1O
2199.0,CC(=O)OC1C(CNC1CC2=CC=C(C=C2)OC)O,1,Cc1ccc(S(=O)(=O)/N=C2\C=C(Nc3ccc(O)cc3)C(=O)c3...
...,...,...,...
135831485.0,CC(=O)C1=CC=C(C=C1)NC(=O)CSC2=NC3=C(C=NN3)C(=O)N2,0,COc1ccc(NC(=O)CSc2nc(O)cc(=O)n2-c2ccccc2OC)cc1
135900555.0,CC(=O)NC1=CC=C(C=C1)NC(=O)CSC2=NC(=C(C(=O)N2)N...,0,COc1ccc(NC(=O)CSc2ncnc3c2cnn3Cc2ccccc2)cc1
135976680.0,C1=CNC(=C1)/C=N/NC(=O)CSC2=C(C=CC(=C2)Cl)Cl,0,CC1(C)CC(=O)C=C(c2ccc3c(c2)OCO3)C1


In [38]:
idx= np.unique(Original_data.index.values, return_index = True )[1]
Original_data =Original_data.iloc[idx]
Original_data.to_csv("./Original_data.csv",sep=',',header=1,index=1)
Canonical_smis=Original_data['Canonical_smiles']
Original_data

,IsomericSMILES,Activity,Canonical_smiles
cid,,,
596.0,C1=CN(C(=O)N=C1N)C2C(C(C(O2)CO)O)O,0,CC(C)CCCCCCCCCCCCC(=O)NCC(=O)NC1C(C)OC(Nc2ncnc...
1318.0,C1=CC2=C(C3=C(C=CC=N3)C=C2)N=C1,1,NC1=C(Cl)C(=O)c2ccc(S(N)(=O)=O)cc2C1=O
1570.0,C1=CC(=S)N(C=C1)O,1,O=C1NC2C(=CC(O)C(O)C2O)c2cc3c(c(O)c21)OCO3
1821.0,C1=C(C(=O)NC(=O)N1C2C(C(C(O2)CO)O)O)F,1,OCC1OC(n2ccc3c(NO)ncnc32)C(O)C1O
2199.0,CC(=O)OC1C(CNC1CC2=CC=C(C=C2)OC)O,1,Cc1ccc(S(=O)(=O)/N=C2\C=C(Nc3ccc(O)cc3)C(=O)c3...
...,...,...,...
135831485.0,CC(=O)C1=CC=C(C=C1)NC(=O)CSC2=NC3=C(C=NN3)C(=O)N2,0,COc1ccc(NC(=O)CSc2nc(O)cc(=O)n2-c2ccccc2OC)cc1
135900555.0,CC(=O)NC1=CC=C(C=C1)NC(=O)CSC2=NC(=C(C(=O)N2)N...,0,COc1ccc(NC(=O)CSc2ncnc3c2cnn3Cc2ccccc2)cc1
135976680.0,C1=CNC(=C1)/C=N/NC(=O)CSC2=C(C=CC(=C2)Cl)Cl,0,CC1(C)CC(=O)C=C(c2ccc3c(c2)OCO3)C1


In [39]:
#AlvaDesc
from alvadesccliwrapper.alvadesc import AlvaDesc
#AlvaDesc Path(Linux default alvaDescCLI location)
aDesc = AlvaDesc('E:\\C++\\alvaDesc\\alvaDescCLI.exe') 

In [40]:
#Calculate the descriotors
aDesc.set_input_SMILES(Canonical_smis.tolist())
if not aDesc.calculate_descriptors('ALL'):
 print('Error: ' + aDesc.get_error())
else:
 Data_gelation=aDesc.get_output()

In [42]:
X_original_data=pd.DataFrame(Data_gelation)
Desc_names_names=aDesc.get_output_descriptors()
X_original_data.columns=Desc_names_names
X_original_data.index=Original_data.index

X_original_data.head()

,MW,AMW,Sv,Se,Sp,Si,Mv,Me,Mp,Mi,...,s1_numAroBonds,s2_numAroBonds,s3_numAroBonds,s4_numAroBonds,s34_size,s34_relSize,s34_phSize,s34_phRelSize,chiralMoment,chiralPhMoment
cid,,,,,,,,,,,,,,,,,,,,,
596.0,575.85,6.398333,50.7866,89.9047,54.3018,103.2543,0.564296,0.998941,0.603353,1.147270,...,0.0,4.0,8.0,0.0,35.7,0.870732,9.900,0.241463,36.410500,9.464711
1318.0,286.71,11.468400,18.4970,26.5637,18.6192,27.9438,0.739880,1.062548,0.744768,1.117752,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.000000
1570.0,307.28,8.779429,23.1858,36.6945,22.7556,39.4553,0.662451,1.048414,0.650160,1.127294,...,0.0,0.0,2.0,6.0,19.0,0.863636,6.625,0.301136,20.285139,6.320154
1821.0,282.29,8.302647,21.2936,35.4617,21.1023,39.1162,0.626282,1.042991,0.620656,1.150476,...,0.0,0.0,0.0,10.0,17.0,0.850000,7.750,0.387500,19.586399,8.618182
2199.0,418.50,8.718750,33.3035,48.6580,34.5683,53.0758,0.693823,1.013708,0.720173,1.105746,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.000000


In [45]:
#Remove descriptors with excessive NA values
Thresh =int(0.8*len(X_original_data))
X_NAomit_data=X_original_data.dropna(
    #thresh=Thresh, 
    axis=1) 
X_NAomit_data.head()
X_NAomit_data.to_csv("./X_NAomit_data.csv",sep=',',header=1,index=1)

In [48]:
#Ranksum test
Ranksums_data=X_NAomit_data
Ranksums_data["Activity"]=Original_data['Activity']
Notgelable=Ranksums_data.loc[Ranksums_data["Activity"] == 0]
Gelable=Ranksums_data.loc[Ranksums_data["Activity"] == 1]
Ranksums_result=[]
for i in list(Ranksums_data.columns):
 NG = Notgelable[i]
 G = Gelable[i]
 Ranksums_caculate=ranksums(NG, G, alternative='two-sided')
 Ranksums_result.append(Ranksums_caculate)
Ranksums_results = pd.DataFrame(Ranksums_result, columns=['Statistics','Pvalues'])
Ranksums_results.index=Ranksums_data.columns#
Ranksums_results = Ranksums_results.drop('Activity',axis = 0)
Ranksums_results.sort_values(by="Pvalues" , inplace=True, ascending=True)
Ranksums_results

,Statistics,Pvalues
SAscore,-4.667245,0.000003
WiA_D/Dt,3.975201,0.000070
J_D/Dt,-3.898537,0.000097
SpMaxA_D/Dt,3.827055,0.000130
Chi_D/Dt,-3.769503,0.000164
...,...,...
B10[C-I],0.000000,1.000000
CATS2D_00_AL,0.000000,1.000000
CATS2D_01_PP,0.000000,1.000000
B10[N-Si],0.000000,1.000000


In [49]:
#Ranksum test <0.05
Ranksums_marked = Ranksums_results[Ranksums_results['Pvalues'] <0.05]
print(Ranksums_marked.shape)
Ranksums_marked.head()

(166, 2)


,Statistics,Pvalues
SAscore,-4.667245,0.000003
WiA_D/Dt,3.975201,0.000070
J_D/Dt,-3.898537,0.000097
SpMaxA_D/Dt,3.827055,0.000130
Chi_D/Dt,-3.769503,0.000164


In [51]:
X_ranksums_data=X_NAomit_data[Ranksums_marked.index]
X_ranksums_data.head()

,SAscore,WiA_D/Dt,J_D/Dt,SpMaxA_D/Dt,Chi_D/Dt,GATS3m,P_VSA_charge_1,NNRS,QED,RCI,...,MPC04,T(N..S),GATS7m,SpMax_D/Dt,EE_D/Dt,SM6_D/Dt,MATS1i,SM5_D/Dt,SpMAD_EA(ri),Eig06_AEA(bo)
cid,,,,,,,,,,,,,,,,,,,,,
596.0,4.865901,0.794751,15.182281,0.797134,1.412305,0.958085,167.184356,0.666667,0.206271,1.133333,...,4.430817,0.0,0.967877,32.682509,32.682509,20.921043,-0.193233,17.434179,1.375080,3.236069
1318.0,3.783433,0.490669,15.767667,0.490019,2.489632,0.887739,32.897187,0.500000,0.707327,1.200000,...,3.871201,6.0,0.933326,8.820334,8.822521,13.062494,0.262703,10.884655,1.512471,2.589960
1570.0,4.474966,0.293714,21.651786,0.289034,4.330357,0.836841,210.743899,0.250000,0.404013,1.352941,...,4.418841,0.0,1.280618,6.358746,6.393760,11.101506,0.117125,9.242200,1.497346,3.097492
1821.0,4.589055,0.487073,13.796170,0.476568,2.508395,0.871307,140.949010,0.666667,0.382417,1.142857,...,4.158883,0.0,1.010979,9.531355,9.532532,13.527554,-0.192423,11.272737,1.398603,2.734293
2199.0,3.920081,0.525362,14.789693,0.523202,2.240863,1.026121,60.694094,0.750000,0.584463,1.090909,...,4.394449,5.0,1.043881,15.696074,15.696078,16.520488,-0.013368,13.767023,1.477469,3.577668


In [52]:
#Scale data
Scaler = preprocessing.MinMaxScaler() #StandardScaler
Transformer =Scaler.fit(X_ranksums_data)
X_scaled_data=Transformer.transform(X_ranksums_data)
X_scaled_data =pd.DataFrame(X_scaled_data)
X_scaled_data.columns=X_ranksums_data.columns
X_scaled_data.index=Original_data.index
X_scaled_data.to_csv("./X_scaled_data.csv",sep=',',header=1,index=1)
joblib.dump(Transformer, './Scaler_transformer.pkl')
X_scaled_data.head()

,SAscore,WiA_D/Dt,J_D/Dt,SpMaxA_D/Dt,Chi_D/Dt,GATS3m,P_VSA_charge_1,NNRS,QED,RCI,...,MPC04,T(N..S),GATS7m,SpMax_D/Dt,EE_D/Dt,SM6_D/Dt,MATS1i,SM5_D/Dt,SpMAD_EA(ri),Eig06_AEA(bo)
cid,,,,,,,,,,,,,,,,,,,,,
596.0,0.483975,0.742006,0.142647,0.775040,0.074656,0.502577,0.299167,0.666667,0.161443,0.498667,...,0.635921,0.000000,0.392684,0.683875,0.681787,0.867268,0.516225,0.867604,0.395336,0.737895
1318.0,0.245427,0.359782,0.155196,0.367516,0.269726,0.462730,0.058868,0.500000,0.728096,0.528000,...,0.460867,0.030457,0.378666,0.126860,0.121143,0.365811,0.842371,0.367367,0.695037,0.590568
1570.0,0.397823,0.112213,0.281331,0.100821,0.603024,0.433900,0.377115,0.250000,0.385073,0.595294,...,0.632174,0.000000,0.519568,0.069399,0.064074,0.240680,0.738235,0.241921,0.662042,0.706296
1821.0,0.422965,0.355262,0.112934,0.349667,0.273123,0.453423,0.252221,0.666667,0.360650,0.502857,...,0.550857,0.000000,0.410171,0.143457,0.137826,0.395487,0.516805,0.397008,0.446648,0.623479
2199.0,0.275540,0.403391,0.134232,0.411549,0.224681,0.541115,0.108609,0.750000,0.589147,0.480000,...,0.624544,0.025381,0.423520,0.287360,0.282653,0.586467,0.644888,0.587516,0.618684,0.815787


In [60]:
#Remove high correlation descriptors
X_cor_data=trimm_correlated(X_scaled_data,0.80)
X_cor_data.index=Original_data.index
X_cor_data.to_csv("./ML_data.csv",sep=',',header=1,index=1)
print_RxC(X_cor_data)
X_cor_data.head()

5506 Moleculars × 58 Descriptors


,SAscore,WiA_D/Dt,J_D/Dt,GATS3m,P_VSA_charge_1,QED,CATS2D_04_LL,JGI8,F04[O-O],HVcpx,...,F02[N-O],SpMax1_Bh(i),P_VSA_m_4,SHED_DD,CATS2D_07_AL,JGI4,ChiA_B(e),GATS7m,MATS1i,SpMAD_EA(ri)
cid,,,,,,,,,,,,,,,,,,,,,
596.0,0.483975,0.742006,0.142647,0.502577,0.299167,0.161443,0.200000,0.334059,0.0625,0.853155,...,0.3,0.552128,0.00000,0.458162,0.051282,0.308730,0.560585,0.392684,0.516225,0.395336
1318.0,0.245427,0.359782,0.155196,0.462730,0.058868,0.728096,0.036364,0.000000,0.0000,0.322976,...,0.2,0.459245,0.22279,0.075591,0.000000,0.551648,0.022526,0.378666,0.842371,0.695037
1570.0,0.397823,0.112213,0.281331,0.433900,0.377115,0.385073,0.000000,0.524238,0.1875,0.373882,...,0.1,0.451751,0.00000,0.340399,0.000000,0.552143,0.203601,0.519568,0.738235,0.662042
1821.0,0.422965,0.355262,0.112934,0.453423,0.252221,0.360650,0.000000,0.494443,0.0625,0.412350,...,0.1,0.612418,0.00000,0.572869,0.051282,0.325000,0.288202,0.410171,0.516805,0.446648
2199.0,0.275540,0.403391,0.134232,0.541115,0.108609,0.589147,0.072727,0.349377,0.0000,0.573399,...,0.2,0.506044,0.02878,0.075591,0.128205,0.379048,0.140084,0.423520,0.644888,0.618684
